In [136]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

In [194]:
load_dotenv()

user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')
db = os.getenv('DB_NAME')

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}")

In [195]:
df = pd.read_sql_table('offers', engine)

# CLEANING

In [198]:
df.isnull().sum()
df_clean = df.drop_duplicates(subset=['job_title','company','city', 'position_level'])
df_clean.to_sql("offers_clean", engine, if_exists="replace", index=False)


354

# Job Title for Junior Spec

In [202]:
query = """
SELECT job_title, position_level, city FROM offers WHERE position_level = 'Młodszy specjalista (Junior)'
"""

df = pd.read_sql_query(query,engine)
df

,job_title,position_level,city
0,"Software Engineer - Early Career, Cloud AI",Młodszy specjalista (Junior),"Warszawa, Śródmieście"
1,Software Engineer II - Ambient Platform,Młodszy specjalista (Junior),"Warszawa, Śródmieście"
2,Junior Data Engineer with SQL,Młodszy specjalista (Junior),Warszawa
3,Junior IT Consultant & Developer,Młodszy specjalista (Junior),"Warszawa, Mokotów"
4,"Junior Software Engineer (Python, Back-End)",Młodszy specjalista (Junior),"Gdynia, Mały Kack"
5,Junior System Integration Test Engineer,Młodszy specjalista (Junior),"Gdynia, Mały Kack"
6,"Software Engineer II - Infrastructure, EasyGCP",Młodszy specjalista (Junior),"Warszawa, Śródmieście"
7,Junior Data Consultant,Młodszy specjalista (Junior),"Warszawa, Wola"
8,Junior Data Scientist (Credit Risk),Młodszy specjalista (Junior),"Warszawa, Wola"
9,Python Junior Developer,Młodszy specjalista (Junior),"Wrocław, Śródmieście"


# The city with the most job offers

In [200]:
query = """
SELECT COUNT(*) AS number_of_jobs,
CASE
    WHEN city LIKE '%%Warszawa%%' THEN 'Warszawa'
    WHEN city LIKE '%%Kraków%%' THEN 'Kraków'
    WHEN city LIKE '%%Poznań%%' THEN 'Poznań'
    WHEN city LIKE '%%Wrocław%%' THEN 'Wrocław'
    WHEN city LIKE '%%Gdynia%%' THEN 'Gdynia'
    WHEN city LIKE '%%Gdańsk%%' THEN 'Gdańsk'
    WHEN city LIKE '%%Białystok%%' THEN 'Białystok'
    ELSE city
END AS city_sum
FROM offers
GROUP BY city_sum
ORDER BY number_of_jobs DESC
LIMIT 1
"""

df = pd.read_sql_query(query, engine)
df

,number_of_jobs,city_sum
0,154,Warszawa


# TOP 20 TECHNOLOGIES except Python

In [201]:
query = """
SELECT 
    TRIM(tech) AS technology,
    COUNT(*) AS occurrences
FROM (
    SELECT unnest(string_to_array(technologies, ',')) AS tech
    FROM offers
    WHERE technologies IS NOT NULL
) AS techs
WHERE TRIM(tech) ILIKE '%%' AND TRIM(tech) NOT ILIKE 'python'
GROUP BY technology
ORDER BY occurrences DESC
LIMIT 20;
"""

df = pd.read_sql_query(query, engine)
df

,technology,occurrences
0,SQL,138
1,Git,54
2,Docker,48
3,Bash,43
4,Java,42
5,JavaScript,42
6,C++,42
7,AWS,38
8,Kubernetes,38
9,Linux,30
